# Ejercicio: creando una red neuronal artificial sencilla con Keras

<img src="https://keras.io/img/logo.png">

En este ejercicio vamos a crear una primera red neuronal artificial con Keras, para resolver un problema sencillo de clasificación binaria.

## Guía general

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema. ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda ejecutar este notebook en uno de los [entornos de Deep Learning recomendados](https://github.com/albarji/teaching-environments-deeplearning), o hacer uso [Google Colaboratory](https://colab.research.google.com/). Si usas Colaboratory, asegúrate de [conectar una GPU](https://colab.research.google.com/notebooks/gpu.ipynb).

El siguiente código mostrará todas las gráficas en el propio notebook en lugar de generar una nueva ventana.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

También vamos a fijar las semillas aleatorias de numpy y tensorflow para obtener resultados reproducibles entre varias ejecuciones del notebook

In [2]:
import numpy as np
import tensorflow as tf
np.random.seed(1)
tf.random.set_seed(2)

Finalmente, si necesitas ayuda en el uso de cualquier función Python, coloca el cursor sobre su nombre y presiona Shift+Tab. Aparecerá una ventana con su documentación. Esto solo funciona dentro de celdas de código.

¡Vamos alla!

## Carga de datos

Vamos a utilizar un problema sencillo de clasificación binaria, que no requiera de un preprocesado de datos complejo: [detección de cáncer de mama en base a unas características precalculadas](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). Este dataset está incluído entre los [datasets de juguete de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer).

In [3]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()

Podemos ver que en este problema contamos con 569 datos en total, cada uno de ellos de 30 variables explicativas, y que solo existen dos clases.

In [4]:
print("Forma de los datos de entrada:", data.data.shape)
print("Categorías a clasificar:", set(data.target))

Forma de los datos de entrada: (569, 30)
Categorías a clasificar: {0, 1}


Ahora separamos un tercio de los datos como conjunto de test, empleando el método [train_test_split de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [5]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=42)

Vamos a estandarizar las variables de entrada para asegurar un buen funcionamiento de la red neuronal. Esto puede hacerse fácilmente con el [StandardScaler de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

## Perceptrón

Empezaremos resolviendo el problema con la red neuronal más sencilla posible: un perceptrón. Esto es básicamente una red neuronal sin capas ocultas, teniendo solo una capa de salida que recibe información directamente de las variables explicativas.

### Definiendo la red

Para construir una red en Keras primero debemos definir el tipo de arquitectura:

* **Sequential**: cada nueva capa se conecta a la capa declarada inmediatamente antes en la red, siguiendo una cadena.
* **Functional**: cada capa puede conectarse a la salida de cualquier otra capa declarada antes en la red, siempre y cuando no se formen ciclos.

Para este ejercicio será suficiente con una arquitectura de tipo Sequential.

In [7]:
from tensorflow.keras import Sequential

perceptron = Sequential()

Una vez la red ha sido inicializada, podemos ir añadiendo las capas deseadas de manera iterativa. Para construir un perceptrón solo nos hará falta una capa "clásica" de pesos desde las entradas a la salida. En Deep Learning moderno este tipo de capas se llaman **Dense**, porque conectan todas las variables de entrada con todas las salidas de la capa.

In [8]:
from tensorflow.keras.layers import Dense

Para crear una capa densa suele bastar con declarar el número de unidades de salida (o desde un punto de vista clásico, el número de neuronas). No obstante, en un modelo de tipo Sequential estamos obligados a declarar el número de variables de entrada a la red neuronal cuando creamos nuestra primera capa. En este problema tenemos 30 variables explicativas, y podemos representar la salida de la red como una única neurona que nos diga si el dato es de clase 0 o 1. Por tanto, nos basta con crear la capa Dense y añadirla al modelo de la siguiente manera.

In [9]:
perceptron.add(Dense(1, input_dim=30))

Como el problema es de clasificación binaria, nos interesa limitar el rango de valores que puede generar la única neurona de salida de la red. Para ello, lo más conveniente es usar la función de activación sigmoidal, la cual podemos añadir de la siguiente manera.

In [10]:
from tensorflow.keras.layers import Activation
perceptron.add(Activation('sigmoid'))

Con esto, la definición de la red está completa. Podemos confirmar que la hemos construído correctamente pidiendo a Keras un resumen del modelo.

In [11]:
perceptron.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 31        
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


### Compilando la red

Tras definir la red, debemos realizar la compilación de la misma. La compilación es un proceso automático que transforma la definición de la red en una formulación simbólica equivalente para la que pueden calcularse las derivadas, permitiendo así ejecutar el algoritmo de retropropagación (backpropagation). Los únicos parámetros que debemos especificar son la función de pérdida o error que la red debe minimizar, y el optimizador a usar durante el aprendizaje.

Dado que estamos tratando con un problema de clasificación binaria, la función de pérdida más adecuada es la **binary crossentropy**. En cuanto al optimizador, de momento utilizaremos el **Stochastic Gradient Descent**. Como parámetro opcional, solicitaremos que como métrica se nos informe de la **accuracy** durante el entrenamiento de la red.

In [12]:
perceptron.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Entrenando la red

Ahora podemos invocar al método `fit` de la red, el encargado de ejecutar el proceso de entrenamiento. Se configura de la siguiente manera

In [13]:
perceptron.fit(
    X_train, # Features de los datos de entrenamiento
    y_train, # Etiquetas de los datos de entrenamiento
    epochs=25, # Número de épocas: las pasadas que hará la red neuronal sobre nuestros datos
    verbose=2, # Nivel de verbosidad de los mensajes
)

Epoch 1/25
12/12 - 2s - loss: 1.1425 - accuracy: 0.3858 - 2s/epoch - 130ms/step
Epoch 2/25
12/12 - 0s - loss: 0.8186 - accuracy: 0.5328 - 73ms/epoch - 6ms/step
Epoch 3/25
12/12 - 0s - loss: 0.6243 - accuracy: 0.6483 - 80ms/epoch - 7ms/step
Epoch 4/25
12/12 - 0s - loss: 0.5043 - accuracy: 0.7428 - 67ms/epoch - 6ms/step
Epoch 5/25
12/12 - 0s - loss: 0.4270 - accuracy: 0.8215 - 67ms/epoch - 6ms/step
Epoch 6/25
12/12 - 0s - loss: 0.3746 - accuracy: 0.8556 - 64ms/epoch - 5ms/step
Epoch 7/25
12/12 - 0s - loss: 0.3373 - accuracy: 0.8635 - 78ms/epoch - 6ms/step
Epoch 8/25
12/12 - 0s - loss: 0.3098 - accuracy: 0.8714 - 64ms/epoch - 5ms/step
Epoch 9/25
12/12 - 0s - loss: 0.2883 - accuracy: 0.8924 - 53ms/epoch - 4ms/step
Epoch 10/25
12/12 - 0s - loss: 0.2713 - accuracy: 0.9003 - 70ms/epoch - 6ms/step
Epoch 11/25
12/12 - 0s - loss: 0.2575 - accuracy: 0.9055 - 70ms/epoch - 6ms/step
Epoch 12/25
12/12 - 0s - loss: 0.2459 - accuracy: 0.9160 - 62ms/epoch - 5ms/step
Epoch 13/25
12/12 - 0s - loss: 0.2359

Ahora que nuestra red está entrenada, podemos obtener predicciones para el conjunto de test como

In [14]:
probs = perceptron.predict(X_test)

Las predicciones se obtienen como una matriz con forma `(n_datos, 1)`, que pueden interpretarse como las probabilidades de clase positiva de cada uno de los datos de test

In [15]:
probs

array([[8.2359213e-01],
       [2.4533404e-02],
       [2.1730235e-01],
       [8.9330715e-01],
       [9.6365744e-01],
       [1.4814468e-04],
       [6.8641081e-04],
       [1.8255050e-01],
       [5.2144456e-01],
       [9.0701187e-01],
       [8.0372345e-01],
       [1.5623750e-01],
       [8.0725110e-01],
       [1.7412093e-01],
       [9.4709617e-01],
       [4.5873248e-03],
       [8.6750227e-01],
       [9.9109578e-01],
       [9.9869812e-01],
       [5.6936899e-03],
       [6.9096822e-01],
       [9.3893313e-01],
       [5.1145529e-04],
       [9.8312843e-01],
       [9.2818588e-01],
       [9.3259591e-01],
       [9.1269159e-01],
       [8.3281672e-01],
       [9.1060787e-01],
       [9.1524562e-03],
       [9.3252212e-01],
       [9.6842813e-01],
       [9.8516709e-01],
       [9.1234899e-01],
       [9.7556520e-01],
       [9.3941903e-01],
       [2.5001132e-01],
       [9.5101076e-01],
       [3.8085695e-02],
       [6.6624218e-01],
       [9.8525214e-01],
       [9.067594

También podemos medir directamente el porcentaje de aciertos de nuestra red sobre el test, usando el método **evaluate**

In [16]:
score = perceptron.evaluate(X_test, y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

6/6 [==============================] - 0s 5ms/step - loss: 0.1635 - accuracy: 0.9628
Test loss 0.1634903848171234
Test accuracy 0.9627659320831299


## Perceptrón multicapa

Pasando a los años 80, podemos mejorar nuestra red introduciendo **capas ocultas**. En Keras podemos hacer esto de forma sencilla introduciendo capas Dense adicionales. Por ejemplo, podemos crear una red con una capa oculta de 64 neuronas de la siguiente manera.

In [17]:
pmc = Sequential()
pmc.add(Dense(64, input_dim=30, activation='sigmoid'))
pmc.add(Dense(1, activation='sigmoid'))

En esta ocasión hemos añadido las activaciones declarándolas en la propia capa Dense, de forma que la notación resulte más compacta. Nótese también que hemos puesto una activación de tipo sigmoidal en la capa oculta, para que la red pueda comportarse de manera no lineal. La red, por tanto, queda de la siguiente manera

In [18]:
pmc.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                1984      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,049
Trainable params: 2,049
Non-trainable params: 0
_________________________________________________________________


<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Compila el perceptrón multicapa y entrénalo con los datos de train. ¿Qué nivel de acierto se obtiene al evaluarlo en test?
</font>

***


In [19]:
pmc.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
pmc.fit(
    X_train, # Features de los datos de entrenamiento
    y_train, # Etiquetas de los datos de entrenamiento
    epochs=25, # Número de épocas: las pasadas que hará la red neuronal sobre nuestros datos
    verbose=2, # Nivel de verbosidad de los mensajes
)
score = pmc.evaluate(X_test, y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

Epoch 1/25
12/12 - 1s - loss: 0.7750 - accuracy: 0.3806 - 934ms/epoch - 78ms/step
Epoch 2/25
12/12 - 0s - loss: 0.6611 - accuracy: 0.6483 - 63ms/epoch - 5ms/step
Epoch 3/25
12/12 - 0s - loss: 0.6011 - accuracy: 0.8241 - 58ms/epoch - 5ms/step
Epoch 4/25
12/12 - 0s - loss: 0.5630 - accuracy: 0.8084 - 61ms/epoch - 5ms/step
Epoch 5/25
12/12 - 0s - loss: 0.5334 - accuracy: 0.8136 - 67ms/epoch - 6ms/step
Epoch 6/25
12/12 - 0s - loss: 0.5088 - accuracy: 0.8294 - 58ms/epoch - 5ms/step
Epoch 7/25
12/12 - 0s - loss: 0.4870 - accuracy: 0.8504 - 64ms/epoch - 5ms/step
Epoch 8/25
12/12 - 0s - loss: 0.4671 - accuracy: 0.8556 - 60ms/epoch - 5ms/step
Epoch 9/25
12/12 - 0s - loss: 0.4493 - accuracy: 0.8766 - 65ms/epoch - 5ms/step
Epoch 10/25
12/12 - 0s - loss: 0.4330 - accuracy: 0.8819 - 59ms/epoch - 5ms/step
Epoch 11/25
12/12 - 0s - loss: 0.4180 - accuracy: 0.8898 - 73ms/epoch - 6ms/step
Epoch 12/25
12/12 - 0s - loss: 0.4040 - accuracy: 0.8924 - 59ms/epoch - 5ms/step
Epoch 13/25
12/12 - 0s - loss: 0.39

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Es probable que añadir una capa oculta no haya mejorado los resultados en test. En las siguientes secciones iremos añadiendo algunas de las recetas de Deep Learning para mejorar el rendimiento de la red.
</font>

***

### Más épocas

Un punto distintivo del método `fit` de Keras es que no reinicia el estado de la red neuronal, sino que continua el entrenamiento. Este comportamiento es diferente al que podemos observar en otras librerías de machine learning más clásico como **scikit-learn**, donde el método `fit` reinicia el entrenamiento del modelo. De este modo, en Keras podemos volver a llamar a `fit` para continuar entrenando durante más épocas.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Realiza una nueva llamada a `fit` sobre la misma red `pmc`, y mide el acierto sobre el conjunto de test.
</font>

***


In [20]:
pmc.fit(
    X_train, # Features de los datos de entrenamiento
    y_train, # Etiquetas de los datos de entrenamiento
    epochs=25, # Número de épocas: las pasadas que hará la red neuronal sobre nuestros datos
    verbose=2, # Nivel de verbosidad de los mensajes
)
score = pmc.evaluate(X_test, y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

Epoch 1/25
12/12 - 0s - loss: 0.2847 - accuracy: 0.9291 - 91ms/epoch - 8ms/step
Epoch 2/25
12/12 - 0s - loss: 0.2794 - accuracy: 0.9318 - 88ms/epoch - 7ms/step
Epoch 3/25
12/12 - 0s - loss: 0.2743 - accuracy: 0.9318 - 72ms/epoch - 6ms/step
Epoch 4/25
12/12 - 0s - loss: 0.2696 - accuracy: 0.9318 - 84ms/epoch - 7ms/step
Epoch 5/25
12/12 - 0s - loss: 0.2651 - accuracy: 0.9318 - 60ms/epoch - 5ms/step
Epoch 6/25
12/12 - 0s - loss: 0.2609 - accuracy: 0.9344 - 65ms/epoch - 5ms/step
Epoch 7/25
12/12 - 0s - loss: 0.2567 - accuracy: 0.9344 - 65ms/epoch - 5ms/step
Epoch 8/25
12/12 - 0s - loss: 0.2528 - accuracy: 0.9370 - 69ms/epoch - 6ms/step
Epoch 9/25
12/12 - 0s - loss: 0.2490 - accuracy: 0.9370 - 62ms/epoch - 5ms/step
Epoch 10/25
12/12 - 0s - loss: 0.2455 - accuracy: 0.9370 - 69ms/epoch - 6ms/step
Epoch 11/25
12/12 - 0s - loss: 0.2420 - accuracy: 0.9396 - 85ms/epoch - 7ms/step
Epoch 12/25
12/12 - 0s - loss: 0.2387 - accuracy: 0.9396 - 79ms/epoch - 7ms/step
Epoch 13/25
12/12 - 0s - loss: 0.2355

Observa que la red ya muestra una accuracy muy alta en la primera época. Esto es porque se está continuando el entrenamiento realizado en la celda anterior. En Keras solo podemos reiniciar el entrenamiento de una red si la volvemos a construir desde cero.

### Más capas

Podemos introducir en la red más de una capa oculta si la configuramos adecuadamente en Keras. Por ejemplo, si queremos introducir 3 capas ocultas cada una de ellas con 64 neuronas, lo haríamos de la siguiente manera

In [21]:
pmc2 = Sequential()
pmc2.add(Dense(64, input_dim=30, activation='sigmoid'))
pmc2.add(Dense(64, activation='sigmoid'))
pmc2.add(Dense(64, activation='sigmoid'))
pmc2.add(Dense(1, activation='sigmoid'))

pmc2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                1984      
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 10,369
Trainable params: 10,369
Non-trainable params: 0
_________________________________________________________________


<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Compila y entrena esta nueva red. ¿Qué nivel de acierto obtienes ahora?
</font>

***


In [22]:
pmc2.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

pmc2.fit(
    X_train, # Features de los datos de entrenamiento
    y_train, # Etiquetas de los datos de entrenamiento
    epochs=25, # Número de épocas: las pasadas que hará la red neuronal sobre nuestros datos
    verbose=2, # Nivel de verbosidad de los mensajes
)
score = pmc2.evaluate(X_test, y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

Epoch 1/25
12/12 - 2s - loss: 0.7050 - accuracy: 0.4908 - 2s/epoch - 130ms/step
Epoch 2/25
12/12 - 0s - loss: 0.6727 - accuracy: 0.6194 - 81ms/epoch - 7ms/step
Epoch 3/25
12/12 - 0s - loss: 0.6625 - accuracy: 0.6194 - 91ms/epoch - 8ms/step
Epoch 4/25
12/12 - 0s - loss: 0.6597 - accuracy: 0.6194 - 82ms/epoch - 7ms/step
Epoch 5/25
12/12 - 0s - loss: 0.6582 - accuracy: 0.6194 - 92ms/epoch - 8ms/step
Epoch 6/25
12/12 - 0s - loss: 0.6579 - accuracy: 0.6194 - 73ms/epoch - 6ms/step
Epoch 7/25
12/12 - 0s - loss: 0.6576 - accuracy: 0.6194 - 99ms/epoch - 8ms/step
Epoch 8/25
12/12 - 0s - loss: 0.6570 - accuracy: 0.6194 - 69ms/epoch - 6ms/step
Epoch 9/25
12/12 - 0s - loss: 0.6568 - accuracy: 0.6194 - 88ms/epoch - 7ms/step
Epoch 10/25
12/12 - 0s - loss: 0.6568 - accuracy: 0.6194 - 95ms/epoch - 8ms/step
Epoch 11/25
12/12 - 0s - loss: 0.6566 - accuracy: 0.6194 - 87ms/epoch - 7ms/step
Epoch 12/25
12/12 - 0s - loss: 0.6559 - accuracy: 0.6194 - 110ms/epoch - 9ms/step
Epoch 13/25
12/12 - 0s - loss: 0.655

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Observa cómo va evolucionando la <b>loss</b> durante el aprendizaje, y compáralo con las redes anteriores. Añadir más capas ocultas sin controlar el desvanecimiento de gradientes puede provocar que la red aprenda muy lentamente.
</font>

***

## Evitando el desvanecimiento de gradientes

Si queremos garantizar que todas las capas de la red aprendan correctamente, necesitamos utilizar alguna medida para evitar el desvanecimiento de gradientes. Una medida simple pero bastante efectiva es sustituir la función de la activación de las capas ocultas por la función Rectificada Lineal (ReLU).

In [23]:
pmc_relu = Sequential()
pmc_relu.add(Dense(64, input_dim=30, activation='relu'))
pmc_relu.add(Dense(64, activation='relu'))
pmc_relu.add(Dense(64, activation='relu'))
pmc_relu.add(Dense(1, activation='sigmoid'))

pmc_relu.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 64)                1984      
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 10,369
Trainable params: 10,369
Non-trainable params: 0
_________________________________________________________________


<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Compila y entrena esta nueva red. ¿Han mejorado los resultados respecto a la red anterior con activaciones sigmoidales?
</font>

***


In [24]:
pmc_relu.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
pmc_relu.fit(
    X_train, # Features de los datos de entrenamiento
    y_train, # Etiquetas de los datos de entrenamiento
    epochs=25, # Número de épocas: las pasadas que hará la red neuronal sobre nuestros datos
    verbose=2, # Nivel de verbosidad de los mensajes
)
score = pmc_relu.evaluate(X_test, y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

Epoch 1/25
12/12 - 1s - loss: 0.6909 - accuracy: 0.5984 - 1s/epoch - 118ms/step
Epoch 2/25
12/12 - 0s - loss: 0.6427 - accuracy: 0.6325 - 69ms/epoch - 6ms/step
Epoch 3/25
12/12 - 0s - loss: 0.5988 - accuracy: 0.6693 - 69ms/epoch - 6ms/step
Epoch 4/25
12/12 - 0s - loss: 0.5580 - accuracy: 0.7454 - 62ms/epoch - 5ms/step
Epoch 5/25
12/12 - 0s - loss: 0.5193 - accuracy: 0.8189 - 81ms/epoch - 7ms/step
Epoch 6/25
12/12 - 0s - loss: 0.4829 - accuracy: 0.8583 - 72ms/epoch - 6ms/step
Epoch 7/25
12/12 - 0s - loss: 0.4486 - accuracy: 0.8871 - 86ms/epoch - 7ms/step
Epoch 8/25
12/12 - 0s - loss: 0.4162 - accuracy: 0.9003 - 77ms/epoch - 6ms/step
Epoch 9/25
12/12 - 0s - loss: 0.3861 - accuracy: 0.9108 - 70ms/epoch - 6ms/step
Epoch 10/25
12/12 - 0s - loss: 0.3579 - accuracy: 0.9318 - 74ms/epoch - 6ms/step
Epoch 11/25
12/12 - 0s - loss: 0.3323 - accuracy: 0.9344 - 87ms/epoch - 7ms/step
Epoch 12/25
12/12 - 0s - loss: 0.3090 - accuracy: 0.9370 - 79ms/epoch - 7ms/step
Epoch 13/25
12/12 - 0s - loss: 0.2879

## Controlando el sobreajuste

Si la diferencia entre el loss o la accuracy de training y test es grande, puede que nuestro modelo esté sobreajustando a los datos de entrenamiento. Podemos añadir a nuestra red un método para controlar el sobreajuste usando capas de Dropout. Por ejemplo, para añadir un Dropout del 50% a todas las capas ocultas podemos hacer lo siguiente

In [25]:
from tensorflow.keras.layers import Dropout

pmc_dropout = Sequential()
pmc_dropout.add(Dense(64, input_dim=30, activation='relu'))
pmc_dropout.add(Dropout(0.5))
pmc_dropout.add(Dense(64, activation='relu'))
pmc_dropout.add(Dropout(0.5))
pmc_dropout.add(Dense(64, activation='relu'))
pmc_dropout.add(Dropout(0.5))
pmc_dropout.add(Dense(1, activation='sigmoid'))

pmc_dropout.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 64)                1984      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Compila y entrena esta nueva red. ¿Han mejorado los resultados?
</font>

***


In [26]:
pmc_dropout.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
pmc_dropout.fit(
    X_train, # Features de los datos de entrenamiento
    y_train, # Etiquetas de los datos de entrenamiento
    epochs=25, # Número de épocas: las pasadas que hará la red neuronal sobre nuestros datos
    verbose=2, # Nivel de verbosidad de los mensajes
)
score = pmc_dropout.evaluate(X_test, y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

Epoch 1/25
12/12 - 2s - loss: 0.7380 - accuracy: 0.5669 - 2s/epoch - 140ms/step
Epoch 2/25
12/12 - 0s - loss: 0.7223 - accuracy: 0.5433 - 46ms/epoch - 4ms/step
Epoch 3/25
12/12 - 0s - loss: 0.6978 - accuracy: 0.5801 - 48ms/epoch - 4ms/step
Epoch 4/25
12/12 - 0s - loss: 0.6344 - accuracy: 0.6115 - 49ms/epoch - 4ms/step
Epoch 5/25
12/12 - 0s - loss: 0.5654 - accuracy: 0.7139 - 48ms/epoch - 4ms/step
Epoch 6/25
12/12 - 0s - loss: 0.5660 - accuracy: 0.7087 - 51ms/epoch - 4ms/step
Epoch 7/25
12/12 - 0s - loss: 0.5631 - accuracy: 0.7165 - 48ms/epoch - 4ms/step
Epoch 8/25
12/12 - 0s - loss: 0.5160 - accuracy: 0.7927 - 53ms/epoch - 4ms/step
Epoch 9/25
12/12 - 0s - loss: 0.4896 - accuracy: 0.7822 - 61ms/epoch - 5ms/step
Epoch 10/25
12/12 - 0s - loss: 0.4811 - accuracy: 0.7953 - 47ms/epoch - 4ms/step
Epoch 11/25
12/12 - 0s - loss: 0.4305 - accuracy: 0.8031 - 49ms/epoch - 4ms/step
Epoch 12/25
12/12 - 0s - loss: 0.4604 - accuracy: 0.7874 - 48ms/epoch - 4ms/step
Epoch 13/25
12/12 - 0s - loss: 0.4246

## Optimizando mejor

Como última mejora, vamos a introducir el optimizador [Adam](https://keras.io/api/optimizers/adam/). Este optimizador hace un uso más eficiente de los gradientes calculados durante el backpropagation, por lo que con el mismo número de épocas suele conseguir mejores resultados. Para configurarlo basta con definir la red de la misma manera, e indicar `optimizer="adam"` en el paso de compilación.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Define una nueva red copiando el diseño de la red anterior, compílala usando Adam como optimizador, entrena y mide el acierto sobre el conjunto de test. ¿Has conseguido mejora?
</font>

***


In [27]:
pmc_adam = Sequential()
pmc_adam.add(Dense(64, input_dim=30, activation='relu'))
pmc_adam.add(Dropout(0.5))
pmc_adam.add(Dense(64, activation='relu'))
pmc_adam.add(Dropout(0.5))
pmc_adam.add(Dense(64, activation='relu'))
pmc_adam.add(Dropout(0.5))
pmc_adam.add(Dense(1, activation='sigmoid'))

pmc_adam.summary()

pmc_adam.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
pmc_adam.fit(
    X_train, # Features de los datos de entrenamiento
    y_train, # Etiquetas de los datos de entrenamiento
    epochs=25, # Número de épocas: las pasadas que hará la red neuronal sobre nuestros datos
    verbose=2, # Nivel de verbosidad de los mensajes
)
score = pmc_adam.evaluate(X_test, y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                1984      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 64)                4160      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_18 (Dense)            (None, 1)                